In [17]:
import os
DATA_DIR = '../data'

# Experiment Config
DF_NAME = 'MathQA'
DIFFICULTY = 'easy'
NUM_OF_SAMPLES = 500
NUM_OF_COT = 40
storage_dir = os.path.join(DATA_DIR, f'Evaluation_CoTs/gpt-3.5-turbo-0125')
file_path = os.path.join(storage_dir, f'{DF_NAME}_{DIFFICULTY}.csv')

In [18]:
import pandas as pd
df = pd.read_csv(file_path)
df

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Instruction Violation_36,CoT_37,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39
0,MathQA_dev,Math,a multiple choice test consists of 4 questions...,c,\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",...,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]"
1,MathQA_dev,Math,a 3 - digit positive integer is chosen at rand...,b,Step 1: Calculate the total number of 3-digit ...,e,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]",...,"[(0, 0)]",Step 1: Understand the problem. We need to fin...,5 / 9,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]"
2,MathQA_dev,Math,if x and y are positive integers and 7 + x + y...,c,\nStep 1: Rearrange the given equation 7 + x +...,c,"[(0, 0)]",\nStep 1: Rearrange the given equation 7 + x +...,a,"[(0, 0)]",...,"[(0, 0)]",\n Step 1: Rearrange the given equation...,e,"[(0, 0)]",\n Step 1: Rearrange the given equation...,c,"[(0, 0)]",\n Step 1: Rearrange the given equation...,c,"[(0, 0)]"
3,MathQA_dev,Math,the hcf and lcm of two numbers m and n are res...,d,"\n To find the values of m and n, we fi...",d,"[(0, 0)]",\n Step 1: Find the relationship betwee...,d,"[(0, 0)]",...,"[(0, 0)]",\n Step 1: Find the relationship betwee...,d,"[(0, 0)]",\n Step 1: Find the relationship betwee...,1/18,"[(0, 0)]",\n Step 1: Find the relationship betwee...,b,"[(0, 0)]"
4,MathQA_dev,Math,"in a kilometer race , a beats b by 48 meters o...",a,\nStep 1: Calculate the speed difference betwe...,a,"[(0, 0)]",\n Step 1: Calculate the speed differen...,b,"[(0, 0)]",...,"[(0, 0)]",\n Let's denote the speed of A as Va an...,b,"[(0, 0)]",\nStep 1: Calculate the speed difference betwe...,c,"[(0, 0)]",\nStep 1: Calculate the speed difference betwe...,d,"[(0, 0)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MathQA_dev,Math,the first doughnut is priced at $ 1 and then i...,b,\n Step 1: Let x be the number of dozen...,b,"[(0, 0)]",Step 1: Let x be the number of dozens purchase...,b,"[(0, 0)]",...,"[(0, 0)]",Step 1: Let x be the number of dozens purchase...,b,"[(0, 0)]",Step 1: Let x be the total number of dozens pu...,b,"[(0, 0)]",\nStep 1: Let's denote the number of individua...,17,"[(0, 0)]"
496,MathQA_dev,Math,"during the first two weeks of january , the to...",e,\nStep 1: Let x be the rainfall during the fir...,e,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",...,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",\n Step 1: Let x be the rainfall during...,e,"[(0, 0)]"
497,MathQA_dev,Math,a train crosses a bridge of length 200 m in 10...,c,\nStep 1: Calculate the speed of the train by ...,c,"[(1, 0)]",\n Step 1: Calculate the speed of the t...,100 m,"[(0, 0)]",...,"[(0, 0)]",\nStep 1: Let the length of the train be x met...,c,"[(0, 1), (0, 1)]",\n Step 1: Let the length of the train ...,b,"[(0, 0)]",Step 1: Let's denote the length of the train a...,b,"[(0, 0)]"
498,MathQA_dev,Math,10 ^ ( 600 ) ã · 10 ^ ( 580 ) = ? The options ...,b,\n Step 1: Apply the rule of exponents ...,e,"[(0, 1), (0, 1)]",Step 1: Apply the rule of exponents which stat...,10 ^ 1180,"[(0, 1), (0, 1)]",...,"[(0, 1), (0, 1)]",\n Step 1: Apply the rule of exponents ...,e,"[(0, 0)]",Step 1: Apply the rule of exponents which stat...,10 ^ 1180,"[(0, 0)]",\n Step 1: Apply the rule of exponents ...,e,"[(0, 1), (0, 1)]"


In [14]:
import os
import pandas
import numpy as np
import pandas as pd
import re
import ast

result_dict = {
    'id':[],
    'correct answer': [],
    'CoT answers':[],
    'length':[],
    'instruction violation':[],
    'internal mistake':[]
}
tmp = []
for col in df.columns:
    if col.startswith('Final Answer_'):
        tmp.append(df[col].to_numpy())
tmp_arr = np.vstack(tmp)
cot_answer_li = tmp_arr.T
# Count Steps
step_count_buffer = []
for col in df:
    if col.startswith('CoT_'):
        cleaned_answers = []
        for entry in df[col]:
            # Extract numbers using regular expression
            steps = re.findall(r'[Ss]tep\s?\d', str(entry))
            if steps:
                # Join all numbers with space if there are multiple numbers
                cleaned_answers.append(len(steps))
            else:
                # If no number is found, replace with 'error'
                cleaned_answers.append(0)
        step_count_buffer.append(cleaned_answers)

step_count = np.array(step_count_buffer).T
# Violate instruction
instruction_buffer = []
for col in df:
    if col.startswith('Instruction Violation_'):
        cleaned_answers = []
        for entry in df[col]:
            # Extract numbers using regular expression
            x = ast.literal_eval(entry)
            cleaned_answers.append(sum([sum(idx) for idx in x]))
        instruction_buffer.append(cleaned_answers)

instruction_error = np.array(instruction_buffer).T

# Internal Mistake mentioned
mistake_buffer = []
for col in df:
    if col.startswith('CoT_'):
        cleaned_answers = []
        for entry in df[col]:
            # Extract numbers using regular expression
            misktake = re.findall(r'(be a mistake)|(be an error)', str(entry))
            if misktake:
                cleaned_answers.append(1)
            else:
                # If no number is found, replace with 'error'
                cleaned_answers.append(0)
        mistake_buffer.append(cleaned_answers)

mistakes = np.array(mistake_buffer).T
assert cot_answer_li.shape == step_count.shape == instruction_error.shape == mistakes.shape

In [15]:
for row in range(len(df)):
    cot_li_sample = cot_answer_li[row]
    length_li_sample = step_count[row]
    IV_li_sample = instruction_error[row]
    mistake_sample = mistakes[row]
    result_dict['id'].append(row)
    result_dict['correct answer'].append(df.iloc[row]['Correct Answer'])
    result_dict['CoT answers'].append(cot_li_sample)
    result_dict['length'].append(length_li_sample)
    result_dict['instruction violation'].append(IV_li_sample)
    result_dict['internal mistake'].append(mistake_sample)


In [16]:
df_final = pd.DataFrame.from_dict(result_dict)
df_final

,id,correct answer,CoT answers,length,instruction violation,internal mistake
0,0,c,"[c, c, c, c, c, c, c, c, c, c, c, c, c, c, c, ...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,b,"[e, e, c, a, e, 5 / 9, c, d, c, c, e, 5 / 9, c...","[4, 3, 3, 4, 3, 4, 3, 3, 3, 6, 3, 3, 3, 3, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,c,"[c, a, c, c, c, d, c, c, c, c, c, c, 7, 6, c, ...","[6, 6, 9, 6, 6, 4, 5, 6, 6, 7, 6, 6, 6, 6, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,d,"[d, d, a, 1 / 18, b, b, b, b, b, d, b, 1 / 18,...","[4, 3, 4, 3, 6, 6, 3, 5, 5, 3, 3, 3, 7, 7, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,a,"[a, b, a, d, b, a, b, cannot be determined, b,...","[4, 6, 5, 3, 8, 3, 6, 3, 3, 3, 2, 3, 4, 6, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
495,495,b,"[b, b, b, b, b, b, b, b, 17, b, 17, 17, b, b, ...","[4, 6, 5, 6, 9, 5, 6, 4, 8, 4, 8, 8, 8, 8, 8, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
496,496,e,"[e, e, e, e, e, e, e, e, e, e, e, e, e, e, e, ...","[5, 5, 4, 5, 5, 5, 5, 5, 5, 4, 4, 3, 4, 3, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
497,497,c,"[c, 100 m, 750 m, a, c, e, c, c, a, a, 400, c,...","[2, 6, 2, 3, 13, 7, 4, 6, 13, 4, 4, 6, 12, 11,...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
498,498,b,"[e, 10 ^ 1180, e, error, e, error, e, e, error...","[3, 3, 1, 0, 3, 0, 4, 4, 0, 3, 4, 3, 3, 1, 3, ...","[2, 2, 0, 10, 0, 10, 2, 0, 10, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
